In [ ]:
#NETWORK - CENTRALITY-SPACE (Readme)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Module conducts the following analysis:

# ---- Computation of network centrality scores (marker for spillover potential)
# ---- Figure 3: Spatial spillover poptential of green H2 offtakers across regions
# ---- Figure 4: Spatial spillover poptential of green H2 offtakers across sectors
# ---- Extended Data Figure 3: Distribution of spillover potential excl. heating

# Module is input for:

# ---- Simulation-policy-intervention

In [ ]:
#SET-UP
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.getenv("PROJ_LIB")

check_and_load <- function(packages) {
  for (pkg in packages) {
    if (!requireNamespace(pkg, quietly = TRUE)) {
      message(paste("Installing missing package:", pkg))
      install.packages(pkg, dependencies = TRUE, repos = "https://cloud.r-project.org")
    }
    if (!(pkg %in% (.packages()))) {
      suppressPackageStartupMessages(library(pkg, character.only = TRUE))
    }
  }
}

# Required libraries
required_packages <- c(
  "dplyr",          # data manipulation
  "data.table",     # fast data tables
  "tidyverse",      # tidy tools (ggplot2, dplyr, tidyr)
  "tibble",         # tibbles
  "jsonlite",       # json handling
  "tidyr",          # tidy reshaping

  "sf",             # spatial data
  "giscoR",         # gisco/eurostat shapes
  "geosphere",      # geodesic distances
  "rnaturalearth",  # natural earth basemaps
  "ggspatial",      # map annotations

  "igraph",         # network analysis
  "Matrix",         # sparse matrices
  "RSpectra",       # fast eigen solvers

  "ggplot2",        # plotting
  "ggsci",          # color scale
  "viridis",        # viridis palettes
  "scales",         # axis scaling
  "ggridges",       # ridge plots
  "forcats",        # factor tools
  "patchwork",      # multi-panel layouts
  "ggnewscale",     # multiple scales
  "ggrepel"         # labels
)

# --- Load all required packages (auto-install if missing) ------------
check_and_load(required_packages)

In [ ]:
#MODULE-SPECIFIC INPUTS AND SETTINGS
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Basemap
world <- ne_countries(scale = "medium", returnclass = "sf")

# EU defined H2 Valleys
valley_coords <- tibble::tibble(
  name = c("HEAVENN", "NAHV", "BalticSeaH2", "IMAGHyNE", "HI2 Valley", "CyLH2Valley"),
  lon = c(6.0, 13.5, 25.0, 4.5, 14.5, -5.0),
  lat = c(53.0, 45.5, 60.0, 45.5, 47.0, 41.8)
)
additional_valleys <- tibble::tibble(
  name = c("GreenHysland2", "TRIERES", "CRAVE-H2", "SH2AMROCK",
           "TH2ICINO", "LuxHyVal", "ZAHYR", "CONVEY", "AdvancedH2Valley",
           "H2tALENT", "HySPARK", "EASTGATEH2V"),
  lon = c(3.0, 22.9, 25.2, -9.0, 9.5, 6.1, 25.6, 9.95, -1.0, -8.0, 21.0, 21.2),
  lat = c(39.6, 37.9, 35.2, 53.3, 45.5, 49.8, 42.4, 57.6, 47.5, 38.0, 52.2, 48.7)
)

valleys_sf <- st_as_sf(valley_coords, coords=c("lon","lat"), crs=4326) %>%
  mutate(type="Large-scale H2 Valley")

additional_valleys_sf <- st_as_sf(additional_valleys, coords=c("lon","lat"), crs=4326) %>%
  mutate(type="Small-scale H2 Valley")

empty_valleys <- valleys_sf[0, ]
empty_additional <- additional_valleys_sf[0, ]


In [ ]:
#DATAFILES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

offtakers <- readRDS("offtakers.rds")
neighbors_matrix <- readRDS("neighbors_matrix.rds")
offtakers_4326 <- st_transform(offtakers, crs = 4326)

world <- rnaturalearth::ne_countries(scale = "medium", returnclass = "sf") %>%
  st_crop(xmin = -15, xmax = 45, ymin = 30, ymax = 75) %>%
  st_transform(crs = 4326)

nuts2_shapefile <- giscoR::gisco_get_nuts(year = 2021, nuts_level = 2, resolution = "20")
if (is.na(st_crs(nuts2_shapefile))) st_crs(nuts2_shapefile) <- 4258
nuts2_shapefile <- st_transform(nuts2_shapefile, crs = 4326)

In [ ]:
#COMPUTE NETWORK CENTRALITY SCORES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Degree Centrality
offtakers_4326$degree <- rescale(colSums(neighbors_matrix), to = c(0, 1))

# Betweenness Centrality
g <- graph_from_adjacency_matrix(neighbors_matrix, mode = "directed", weighted = NULL)
offtakers_4326$betweenness <- rescale(betweenness(g, directed = TRUE, normalized = FALSE), to = c(0, 1))

# Eigenvector Centrality (legacy, not used in analysis)
W_sym <- forceSymmetric(neighbors_matrix, uplo = "U")
eig <- RSpectra::eigs(W_sym, k = 1, which = "LM")
eig_vector <- Re(eig$vectors[, 1])
deg <- colSums(neighbors_matrix)
if (cor(eig_vector, deg) < 0) eig_vector <- -eig_vector
offtakers_4326$eigen_centrality <- rescale(eig_vector, to = c(0, 1))

# Add to projected df
offtakers <- offtakers%>%
  left_join(
    offtakers_4326 %>%
      st_drop_geometry() %>%
      select(plant_id, eigen_centrality, degree, betweenness),
    by = "plant_id"
  )

#Save datfile
saveRDS(offtakers, "offtakers_centrality.rds")


#Reproject to long
offtakers_long <- offtakers %>%
  st_transform(4326) %>%
  select(plant_id, geometry, degree, eigen_centrality, betweenness) %>%
  tidyr::pivot_longer(
    cols = c(degree, eigen_centrality, betweenness),
    names_to = "centrality_type",
    values_to = "centrality_score"
  ) %>%
  mutate(alpha_value = ifelse(centrality_score < 0.05, 0.1, 0.5)) 

In [ ]:
#TOP NUTS-2 REGIONS BY SHARE OF TOP OFFTAKERS (TEXT)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Identify global top-10% offtakers for degree and betweenness centrality
nuts2_shapefile <- st_transform(nuts2_shapefile, 3035)
offtakers_nuts2 <- offtakers %>%
  st_join(nuts2_shapefile %>% select(NUTS_ID))  

# Long table of scores
cent_long <- offtakers_nuts2 %>%
  st_drop_geometry() %>%
  select(NUTS_ID, degree, eigen_centrality, betweenness) %>%
  tidyr::pivot_longer(
    cols = c(degree, eigen_centrality, betweenness),
    names_to = "metric", values_to = "score"
  )

top10_global_subset <- cent_long %>%
  filter(metric %in% c("degree", "betweenness")) %>%   # keep only these two
  group_by(metric) %>%
  mutate(
    cutoff = quantile(score, 0.9, na.rm = TRUE),
    is_top10 = score >= cutoff
  ) %>%
  ungroup()

# Count how many top offtakers per NUTS2

nuts2_top_shares_subset <- top10_global_subset %>%
  filter(is_top10) %>%                     
  group_by(metric, NUTS_ID) %>%
  summarise(n_top = n(), .groups = "drop") %>%
  group_by(metric) %>%
  mutate(
    total_top = sum(n_top),
    share_pct = 100 * n_top / total_top
  ) %>%
  ungroup() %>%
  left_join(
    st_drop_geometry(nuts2_shapefile) %>% select(NUTS_ID, NAME_LATN),
    by = "NUTS_ID"
  ) %>%
  arrange(metric, desc(share_pct))

nuts2_top_shares_subset

In [ ]:
#TOP NUTS-2 REGIONS BY SHARE OF TOP OFFTAKERS (TEXT)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Identify top-10% offtakers by metric
cent_long_subset <- cent_long %>%
  filter(metric %in% c("degree", "betweenness"))
top10_subset <- cent_long_subset %>%
  group_by(metric) %>%
  mutate(
    cutoff   = quantile(score, 0.9, na.rm = TRUE),
    is_top10 = score >= cutoff
  ) %>%
  ungroup()

# Counts and shares by NUTS2-region
nuts2_top_stats <- top10_subset %>%
  filter(is_top10) %>%                     
  group_by(metric, NUTS_ID) %>%
  summarise(n_top = n(), .groups = "drop") %>%
  group_by(metric) %>%
  mutate(
    total_top = sum(n_top),
    share_pct = 100 * n_top / total_top
  ) %>%
  ungroup()

# Add region clear names
nuts2_top_stats_named <- nuts2_top_stats %>%
  left_join(
    st_drop_geometry(nuts2_shapefile) %>% select(NUTS_ID, NAME_LATN),
    by = "NUTS_ID"
  )

# Add cumulative shares, sort descending

nuts2_top_stats_cumulative <- nuts2_top_stats_named %>%
  arrange(metric, desc(share_pct)) %>%
  group_by(metric) %>%
  mutate(
    cumulative_share_pct = cumsum(share_pct)
  ) %>%
  ungroup() %>%
  select(metric, NAME_LATN, n_top, share_pct, cumulative_share_pct)

# Print

nuts2_top_stats_cumulative


In [ ]:
#FIGURE 3: DISTRIBUTION OF SPILLOVER POTENTIAL BY REGION
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Plot settings and bouding box
        pop_max_cutoff <- 250000               #Cities to be annotated in plot
        highlight <- 50                        #Offtakers to be highlighted
        common_xlim <- c(-15, 45)              #Xlimits
        common_ylim <- c(30, 75)               #Ylimits
        mean_lat_global <- mean(common_ylim)   #Meanlat  
        deg2rad <- function(d) d * pi / 180    #Deg2rad
        
        # Zoom1: NRW
        zoom1_center <- c(lon =7.0, lat = 51.5)
        zoom1_height <- 2
        zoom1_width <- zoom1_height *
          (diff(common_xlim) * cos(deg2rad(mean_lat_global))) /
          (diff(common_ylim) * cos(deg2rad(zoom1_center["lat"])))
        zoom1_xlim <- c(zoom1_center["lon"] - zoom1_width/2,
                        zoom1_center["lon"] + zoom1_width/2)
        zoom1_ylim <- c(zoom1_center["lat"] - zoom1_height/2,
                        zoom1_center["lat"] + zoom1_height/2)
        
        # Zoom2: AT/CZ/SK
        zoom2_center <- c(lon = 16.5, lat = 48.2)
        zoom2_height <- 3
        zoom2_width <- zoom2_height *
          (diff(common_xlim) * cos(deg2rad(mean_lat_global))) /
          (diff(common_ylim) * cos(deg2rad(zoom2_center["lat"])))
        zoom2_xlim <- c(zoom2_center["lon"] - zoom2_width/2,
                        zoom2_center["lon"] + zoom2_width/2)
        zoom2_ylim <- c(zoom2_center["lat"] - zoom2_height/2,
                        zoom2_center["lat"] + zoom2_height/2)

# Offtaker data
        offtakers_long <- offtakers %>%
          st_transform(4326) %>%
          select(plant_id, installation_name, geometry, sector,
                 degree, eigen_centrality, betweenness) %>%
          pivot_longer(
            cols=c(degree, eigen_centrality, betweenness),
            names_to="centrality_type",
            values_to="centrality_score"
          )


# Helper functions
        make_bbox_polygon <- function(xmin, xmax, ymin, ymax, crs = 4326) {
          coords <- matrix(
            c(xmin, ymin,
              xmin, ymax,
              xmax, ymax,
              xmax, ymin,
              xmin, ymin),
            ncol = 2, byrow = TRUE
          )
          st_sfc(st_polygon(list(coords)), crs = crs)
        }
        bbox_zoom1 <- make_bbox_polygon(zoom1_xlim[1], zoom1_xlim[2],
                                        zoom1_ylim[1], zoom1_ylim[2])
        bbox_zoom2 <- make_bbox_polygon(zoom2_xlim[1], zoom2_xlim[2],
                                        zoom2_ylim[1], zoom2_ylim[2])
        
        get_topN_zoom <- function(data, metric, bbox, n=highlight) {
          data %>%
            filter(centrality_type == metric) %>%
            st_intersection(bbox) %>%
            arrange(desc(centrality_score)) %>%
            slice_head(n=n)
        }
        
        active_deg1 <- get_topN_zoom(offtakers_long, "degree", bbox_zoom1) %>%
          st_drop_geometry() %>% pull(sector) %>% unique()
        
        active_bet2 <- get_topN_zoom(offtakers_long, "betweenness", bbox_zoom2) %>%
          st_drop_geometry() %>% pull(sector) %>% unique()

# Color and legend
        # Color palette for sectors
        nrc_colors <- pal_npg("nrc")(12)
        sector_levels <- sort(unique(offtakers$sector))
        sector_palette <- setNames(rep(nrc_colors, length.out=length(sector_levels)), sector_levels)
        
        # Special color settings (for improved visibility)
        if ("Other" %in% names(sector_palette)) {
          sector_palette["Other"] <- "#808080"
        }
        if ("Iron & steel" %in% names(sector_palette)) {
          sector_palette["Iron & steel"] <- nrc_colors[1]
        }
        if ("Non-metallic minerals" %in% names(sector_palette)) {
          sector_palette["Non-metallic minerals"] <- nrc_colors[5]
        }
        
        offtakers_long$sector <- factor(offtakers_long$sector, levels=sector_levels)
        
        # City annotations
        cities <- ne_download(scale=10, type="populated_places",
                              category="cultural", returnclass="sf")
        cities_zoom1 <- st_intersection(cities, bbox_zoom1) %>% filter(POP_MAX > pop_max_cutoff)
        cities_zoom2 <- st_intersection(cities, bbox_zoom2) %>% filter(POP_MAX > pop_max_cutoff)
        
        
        # Legend: Global ring sectors
        ring_sectors_global <- union(active_deg1, active_bet2)
        
        # Legend: Dummy data
        legend_df <- data.frame(
          sector = sector_levels,
          is_ring = sector_levels %in% ring_sectors_global,
          x = 1000,
          y = 1000
        )

# Plot function
        plot_centrality_simple <- function(data, metric, title, valleys, valleys_small,
                                           show_legend = FALSE, bbox = NULL) {
        
          ggplot() +
            geom_sf(data=world, fill="grey97", color="grey85", size=0.2) +
            geom_sf(data=filter(data, centrality_type==metric),
                    aes(fill=centrality_score),
                    shape=21, size=0.6, stroke=0, alpha=0.5) +
            geom_sf(data=valleys, aes(shape=type, color=type),
                    size=3.5, stroke=1.2) +
            geom_sf(data=valleys_small, aes(shape=type, color=type),
                    size=2.5, stroke=1.0) +
            {if(!is.null(bbox)) geom_sf(data=bbox, fill=NA, color="black", linewidth=1)} +
            scale_fill_viridis_c(option="mako", direction=-1, limits=c(0,1),
                                 guide=if(show_legend) "colorbar" else "none") +
            scale_shape_manual(values=c("Large-scale H2 Valley"=4,
                                        "Small-scale H2 Valley"=4)) +
            scale_color_manual(values=c("Large-scale H2 Valley"="maroon",
                                        "Small-scale H2 Valley"="maroon")) +
            coord_sf(xlim=common_xlim, ylim=common_ylim, expand=FALSE) +
            labs(title=title) +
            theme_classic(base_size=14) +
            theme(
              plot.title = element_text(hjust=0),
              legend.position = if(show_legend) "right" else "none",
              panel.border = element_rect(color="black", fill=NA, linewidth=0.6)
            )
        }
        
        plot_centrality_sector <- function(data, metric, title,
                                           valleys, valleys_small,
                                           show_legend = FALSE, zoom = FALSE,
                                           zoom_xlim=NULL, zoom_ylim=NULL,
                                           cities_zoom=NULL) {
        
          # subset
          data_sub <- data %>% filter(centrality_type == metric)
        
          # compute topN for zoom
          topN <- get_topN_zoom(data, metric,
                                make_bbox_polygon(zoom_xlim[1], zoom_xlim[2],
                                                  zoom_ylim[1], zoom_ylim[2]))
        
          data_sub <- data_sub %>%
            mutate(is_topN = plant_id %in% topN$plant_id)
        
          ggplot() +
            geom_sf(data=world, fill="grey97", color="grey85", size=0.2) +
            geom_sf(data=data_sub, aes(fill=centrality_score),
                    shape=16, size=2, stroke=0, alpha=0.5) +
            geom_sf(data=topN, aes(fill=centrality_score),
                    shape=21, size=7, stroke=0, alpha=0.3) +
            geom_sf(data=topN, aes(color=sector),
                    shape=21, size=7, stroke=2, fill=NA) +
        
            # unified legend
            geom_point(data=legend_df,
                       aes(x=x, y=y, color=sector),
                       shape=16, size=3, inherit.aes=FALSE) +
            geom_point(data=legend_df %>% filter(is_ring),
                       aes(x=x, y=y, color=sector),
                       shape=21, fill=NA, size=7, stroke=2,
                       inherit.aes=FALSE) +
        
            {if(zoom) geom_sf(data=cities_zoom, color="black", size=1.2)} +
            {if(zoom) geom_label_repel(data=cities_zoom,
                                       aes(label=NAME, geometry=geometry),
                                       stat="sf_coordinates",
                                       size=4, fill="white",
                                       box.padding=0.4, label.size=0.2)} +
        
            scale_fill_viridis_c(option="mako", direction=-1, limits=c(0,1),
                                 guide="none") +
            scale_color_manual(name="Top 50 offtakers / Sector",
                               values=sector_palette,
                               breaks=sector_levels,
                               drop=FALSE,
                               guide=if(show_legend) "legend" else "none") +
        
            coord_sf(xlim=zoom_xlim, ylim=zoom_ylim, expand=FALSE) +
            labs(title=title) +
            theme_classic(base_size=16) +
            theme(
              plot.title = element_text(hjust=0),
              legend.position = if(show_legend) "right" else "none",
              panel.border = element_rect(color="black", fill=NA, linewidth=0.6)
            )
        }

# Plots
        p1 <- plot_centrality_simple(offtakers_long, "degree",
                                     "Local Spillover Potential\n(Degree Centrality)",
                                     valleys_sf, additional_valleys_sf,
                                     bbox=bbox_zoom1)
        
        p2 <- plot_centrality_simple(offtakers_long, "betweenness",
                                     "Spillover Channels\n(Betweenness Centrality)",
                                     empty_valleys, empty_additional,
                                     show_legend=TRUE, bbox=bbox_zoom2)
        
        p3 <- plot_centrality_sector(offtakers_long, "degree",
                                     "North Rhine-Westphalia",
                                     valleys_sf, additional_valleys_sf,
                                     zoom=TRUE, show_legend=FALSE,
                                     zoom_xlim=zoom1_xlim, zoom_ylim=zoom1_ylim,
                                     cities_zoom=cities_zoom1)
        
        p4 <- plot_centrality_sector(offtakers_long, "betweenness",
                                     "Eastern Austria / Moravia",
                                     valleys_sf, additional_valleys_sf,
                                     zoom=TRUE, show_legend=TRUE,
                                     zoom_xlim=zoom2_xlim, zoom_ylim=zoom2_ylim,
                                     cities_zoom=cities_zoom2)
        
        figure3 <- (p1 + p2) / (p3 + p4) +
          plot_annotation(tag_levels="a",
                          theme=theme(plot.tag=element_text(face="bold", size=14))) &
          theme(
            legend.position="right",
            legend.box.spacing = unit(-25,"cm"),
            panel.spacing = unit(0,"cm"),
            axis.text = element_blank(),
            axis.title = element_blank(),
            axis.ticks = element_blank(),
            plot.margin = margin(0,0,0,0)
          )
        
        figure3 <- figure2 + plot_layout(
          guides="collect",
          widths=c(1,0.05)
        )
        
        options(repr.plot.width=24, repr.plot.height=12, repr.plot.res=600)
        print(figure2)
                               
        ggsave("figure3.png", figure3,
               device="png", width=24, height=12,
               units="in", dpi=600, limitsize=FALSE)

In [ ]:
#FIGURE 4: DISTRIBUTION OF SPILLOVER POTENTIAL BY SECTOR
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Plot Theme
style_template <- theme_minimal(base_size = 22) +
  theme(
    axis.line          = element_line(color = "black"),
    axis.ticks         = element_line(color = "black"),
    axis.text          = element_text(size = 22, color = "black"),
    axis.title         = element_text(size = 22, color = "black"),
    plot.title         = element_text(size = 22, hjust = 0.5, color = "black"),
    axis.text.x.top    = element_blank(),
    axis.text.y.right  = element_blank(),
    axis.ticks.x.top   = element_blank(),
    axis.ticks.y.right = element_blank(),
    panel.grid.minor   = element_blank(),
    panel.grid.major   = element_blank(),  
    legend.text        = element_text(size = 22),
    legend.title       = element_text(size = 22)
  )


#Data
centrality_long <- offtakers_4326 %>%
  st_drop_geometry() %>%
  select(sector,
         Degree = degree,
         Betweenness = betweenness) %>%
  pivot_longer(-sector,
               names_to = "centrality_type",
               values_to = "centrality_value") %>%
  mutate(centrality_type = factor(centrality_type,
                                  levels = c("Degree", "Betweenness")))

sector_stats <- offtakers_4326 %>%
  st_drop_geometry() %>%
  group_by(sector) %>%
  summarise(n_offtakers = n(),
            median_outdeg = median(degree, na.rm = TRUE),
            .groups = "drop")

transport_power_heat <- c("Heat", "Power", "Shipping", "Heavy duty", "Aviation")

industrials <- sort(setdiff(sector_stats$sector, transport_power_heat))
transport_ph <- sort(transport_power_heat)

sector_order <- c(
  industrial = industrials,
  tph        = transport_ph
) |> unname()

sector_stats <- sector_stats %>%
  mutate(y_axis_label = factor(sector, levels = rev(sector_order)))
centrality_long <- centrality_long %>%
  mutate(y_axis_label = factor(sector, levels = rev(sector_order)))

#Percentile calculation
percentiles_df <- centrality_long %>%
  group_by(centrality_type) %>%
  summarise(p90 = quantile(centrality_value, 0.9, na.rm = TRUE),
            .groups = "drop")

#Bubbles (threshold)
above_threshold <- centrality_long %>%
  left_join(percentiles_df, by = "centrality_type") %>%
  group_by(y_axis_label, centrality_type) %>%
  summarise(share_above = mean(centrality_value > p90, na.rm = TRUE),
            .groups = "drop") %>%
  mutate(x_pos = 1.12,
         label = scales::percent(share_above, accuracy = 1))

#Legend
transport_sectors   <- c("Aviation", "Shipping", "Heavy duty")
exclude_industrials <- c("Power", "Heat")

centrality_industrial <- centrality_long %>%
  filter(!(sector %in% c(transport_sectors, exclude_industrials)))
centrality_other <- centrality_long %>%
  filter(sector %in% c(transport_sectors, exclude_industrials))

sector_stats <- sector_stats %>%
  mutate(group = ifelse(sector %in% centrality_industrial$sector,
                        "Industrials", "Transport / Heat / Power"))

y_levels <- levels(sector_stats$y_axis_label)
y_scale  <- scale_y_discrete(limits = y_levels, expand = expansion(mult = c(0.01, 0.05)))
ylims_cart <- c(0.5, length(y_levels) + 0.5)

#Bar plot
        xmax_val <- 8000 #X axis setting
        
        bar_plot <- ggplot(sector_stats, aes(x = n_offtakers, y = y_axis_label, fill = group)) +
          geom_col(width = 0.7) +
          geom_vline(xintercept = xmax_val, color = "black", linewidth = 0.6) +
          scale_fill_manual(values = c("Industrials" = "#416AA6",
                                       "Transport / Heat / Power" = "grey70"),
                            name = "Sector") +
          labs(x = "", y = NULL, title = "# Offtakers") +
          scale_x_continuous(breaks = scales::pretty_breaks(),
                             labels = scales::label_number(accuracy = 1),
                             sec.axis = dup_axis(name = NULL, labels = NULL),
                             expand = c(0, 0)) +
          y_scale +
          coord_cartesian(xlim = c(0, xmax_val), ylim = ylims_cart, clip = "off") +
          style_template +
          theme(axis.line.y.right  = element_line(color = "black"),
                axis.ticks.y.right = element_blank(),
                axis.text.y.right  = element_blank(),
                axis.text.y        = element_text(hjust = 0, size = 22, color = "black"))

# Ridge plot
        ridge_plot <- function(type, title, xlab) {
          p90_val <- filter(percentiles_df, centrality_type == type)$p90
          
          ggplot() +
            geom_rect(aes(xmin = p90_val, xmax = 1.2, ymin = -Inf, ymax = Inf),
                      inherit.aes = FALSE, fill = "#EAE6F4", alpha = 0.4) +
            geom_density_ridges_gradient(data = filter(centrality_other, centrality_type == type),
                                         aes(x = centrality_value, y = y_axis_label, fill = ..x..),
                                         scale = 0.8, trim = TRUE, rel_min_height = 0.01,
                                         gradient_lwd = 0.3, color = "black") +
            scale_fill_gradient(low = "grey90", high = "grey20", guide = "none") +
            new_scale_fill() +
            geom_density_ridges_gradient(data = filter(centrality_industrial, centrality_type == type),
                                         aes(x = centrality_value, y = y_axis_label, fill = ..x..),
                                         scale = 0.8, trim = TRUE, rel_min_height = 0.01,
                                         gradient_lwd = 0.3, color = "black") +
            scale_fill_viridis_c(name = "Centrality", option = "mako", direction = -1, end = 0.9) +
            geom_point(data = filter(above_threshold, centrality_type == type),
                       aes(x = x_pos, y = y_axis_label,
                           shape = "Share above\n90th percentile",
                           stroke = share_above * 5),
                       inherit.aes = FALSE, size = 16, fill = "white", color = "black") +
            geom_text(data = filter(above_threshold, centrality_type == type),
                      aes(x = x_pos, y = y_axis_label, label = label),
                      inherit.aes = FALSE, size = 5.6, color = "black",
                      hjust = 0.5, vjust = 0.5) +
            geom_vline(data = filter(percentiles_df, centrality_type == type),
                       aes(xintercept = p90, linetype = "90% percentile"),
                       color = "black", linewidth = 0.6) +
            geom_vline(xintercept = 1.2, color = "black", linewidth = 0.6) +
            scale_x_continuous(limits = c(0, 1.2),
                               expand = c(0, 0),
                               sec.axis = dup_axis(name = NULL, labels = NULL),
                               labels = function(x) ifelse(x == 0, "0", as.character(x))) +
            y_scale +
            scale_shape_manual("", values = c("Share above\n90th percentile" = 21)) +
            scale_linetype_manual("", values = c("90% percentile" = "solid")) +
            labs(x = xlab, y = NULL, title = title) +
            coord_cartesian(ylim = ylims_cart, clip = "off") +
            style_template +
            theme(axis.text.y  = element_blank(),
                  axis.ticks.y = element_blank(),
                  axis.title.y = element_blank(),
                  axis.line.y.right = element_line(color = "black"))
        }
        
        ridge_degree  <- ridge_plot("Degree", "Degree Centrality (H2 valleys)", "Degree centrality")
        ridge_between <- ridge_plot("Betweenness", "Betweenness Centrality (H2 Corridors)", "Betweenness centrality")

#Combine
figure4 <- bar_plot + ridge_degree + ridge_between +
  plot_annotation(tag_levels = "a",
                  theme = theme(plot.tag = element_text(face = "bold", size = 18),
                                plot.tag.position = c(0, 1))) +
  plot_layout(widths = c(0.5, 1, 1), guides = "collect")

options(repr.plot.width = 22, repr.plot.height = 10, repr.plot.res = 600)

figure4

ggsave("figure3.pdf", figure4, device = cairo_pdf, width = 22, height = 10, units = "in", dpi = 600)

In [ ]:
#EXTENDED DATA FIGURE 3: DISTRIBUTION OF SPILLOVER POTRNTIAL BY REGION EXCLUDING HEATING
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Sector exclusion

excluded_sector <- "Heat"

# Plot settings and bouding box
        pop_max_cutoff <- 250000               #Cities to be annotated in plot
        highlight <- 50                        #Offtakers to be highlighted
        common_xlim <- c(-15, 45)              #Xlimits
        common_ylim <- c(30, 75)               #Ylimits
        mean_lat_global <- mean(common_ylim)   #Meanlat  
        deg2rad <- function(d) d * pi / 180    #Deg2rad
        
        # Zoom1: NRW
        zoom1_center <- c(lon =7.0, lat = 51.5)
        zoom1_height <- 2
        zoom1_width <- zoom1_height *
          (diff(common_xlim) * cos(deg2rad(mean_lat_global))) /
          (diff(common_ylim) * cos(deg2rad(zoom1_center["lat"])))
        zoom1_xlim <- c(zoom1_center["lon"] - zoom1_width/2,
                        zoom1_center["lon"] + zoom1_width/2)
        zoom1_ylim <- c(zoom1_center["lat"] - zoom1_height/2,
                        zoom1_center["lat"] + zoom1_height/2)
        
        # Zoom2: AT/CZ/SK
        zoom2_center <- c(lon = 16.5, lat = 48.2)
        zoom2_height <- 3
        zoom2_width <- zoom2_height *
          (diff(common_xlim) * cos(deg2rad(mean_lat_global))) /
          (diff(common_ylim) * cos(deg2rad(zoom2_center["lat"])))
        zoom2_xlim <- c(zoom2_center["lon"] - zoom2_width/2,
                        zoom2_center["lon"] + zoom2_width/2)
        zoom2_ylim <- c(zoom2_center["lat"] - zoom2_height/2,
                        zoom2_center["lat"] + zoom2_height/2)

# Offtaker data
        offtakers_long <- offtakers %>%
          st_transform(4326) %>%
          filter(sector != excluded_sector) %>%
          select(plant_id, installation_name, geometry, sector,
                 degree, eigen_centrality, betweenness) %>%
          pivot_longer(
            cols=c(degree, eigen_centrality, betweenness),
            names_to="centrality_type",
            values_to="centrality_score"
          )

# Helper functions
        make_bbox_polygon <- function(xmin, xmax, ymin, ymax, crs = 4326) {
          coords <- matrix(
            c(xmin, ymin,
              xmin, ymax,
              xmax, ymax,
              xmax, ymin,
              xmin, ymin),
            ncol = 2, byrow = TRUE
          )
          st_sfc(st_polygon(list(coords)), crs = crs)
        }
        bbox_zoom1 <- make_bbox_polygon(zoom1_xlim[1], zoom1_xlim[2],
                                        zoom1_ylim[1], zoom1_ylim[2])
        bbox_zoom2 <- make_bbox_polygon(zoom2_xlim[1], zoom2_xlim[2],
                                        zoom2_ylim[1], zoom2_ylim[2])
        
        get_topN_zoom <- function(data, metric, bbox, n=highlight) {
          data %>%
            filter(centrality_type == metric) %>%
            st_intersection(bbox) %>%
            arrange(desc(centrality_score)) %>%
            slice_head(n=n)
        }
        
        active_deg1 <- get_topN_zoom(offtakers_long, "degree", bbox_zoom1) %>%
          st_drop_geometry() %>% pull(sector) %>% unique()
        
        active_bet2 <- get_topN_zoom(offtakers_long, "betweenness", bbox_zoom2) %>%
          st_drop_geometry() %>% pull(sector) %>% unique()

# Color and legend
        # Color palette for sectors
        nrc_colors <- pal_npg("nrc")(12)
        sector_levels <- sort(unique(offtakers$sector))
        sector_palette <- setNames(rep(nrc_colors, length.out=length(sector_levels)), sector_levels)
        
        # Special color settings (for improved visibility)
        if ("Other" %in% names(sector_palette)) {
          sector_palette["Other"] <- "#808080"
        }
        if ("Iron & steel" %in% names(sector_palette)) {
          sector_palette["Iron & steel"] <- nrc_colors[1]
        }
        if ("Non-metallic minerals" %in% names(sector_palette)) {
          sector_palette["Non-metallic minerals"] <- nrc_colors[5]
        }
        
        offtakers_long$sector <- factor(offtakers_long$sector, levels=sector_levels)
        
        # City annotations
        cities <- ne_download(scale=10, type="populated_places",
                              category="cultural", returnclass="sf")
        cities_zoom1 <- st_intersection(cities, bbox_zoom1) %>% filter(POP_MAX > pop_max_cutoff)
        cities_zoom2 <- st_intersection(cities, bbox_zoom2) %>% filter(POP_MAX > pop_max_cutoff)
        
        
        # Legend: Global ring sectors
        ring_sectors_global <- union(active_deg1, active_bet2)
        
        # Legend: Dummy data
        legend_df <- data.frame(
          sector = sector_levels,
          is_ring = sector_levels %in% ring_sectors_global,
          x = 1000,
          y = 1000
        )

# Plot function
        plot_centrality_simple <- function(data, metric, title, valleys, valleys_small,
                                           show_legend = FALSE, bbox = NULL) {
        
          ggplot() +
            geom_sf(data=world, fill="grey97", color="grey85", size=0.2) +
            geom_sf(data=filter(data, centrality_type==metric),
                    aes(fill=centrality_score),
                    shape=21, size=0.6, stroke=0, alpha=0.5) +
            geom_sf(data=valleys, aes(shape=type, color=type),
                    size=3.5, stroke=1.2) +
            geom_sf(data=valleys_small, aes(shape=type, color=type),
                    size=2.5, stroke=1.0) +
            {if(!is.null(bbox)) geom_sf(data=bbox, fill=NA, color="black", linewidth=1)} +
            scale_fill_viridis_c(option="mako", direction=-1, limits=c(0,1),
                                 guide=if(show_legend) "colorbar" else "none") +
            scale_shape_manual(values=c("Large-scale H2 Valley"=4,
                                        "Small-scale H2 Valley"=4)) +
            scale_color_manual(values=c("Large-scale H2 Valley"="maroon",
                                        "Small-scale H2 Valley"="maroon")) +
            coord_sf(xlim=common_xlim, ylim=common_ylim, expand=FALSE) +
            labs(title=title) +
            theme_classic(base_size=14) +
            theme(
              plot.title = element_text(hjust=0),
              legend.position = if(show_legend) "right" else "none",
              panel.border = element_rect(color="black", fill=NA, linewidth=0.6)
            )
        }
        
        plot_centrality_sector <- function(data, metric, title,
                                           valleys, valleys_small,
                                           show_legend = FALSE, zoom = FALSE,
                                           zoom_xlim=NULL, zoom_ylim=NULL,
                                           cities_zoom=NULL) {
        
          # subset
          data_sub <- data %>% filter(centrality_type == metric)
        
          # compute topN for zoom
          topN <- get_topN_zoom(data, metric,
                                make_bbox_polygon(zoom_xlim[1], zoom_xlim[2],
                                                  zoom_ylim[1], zoom_ylim[2]))
        
          data_sub <- data_sub %>%
            mutate(is_topN = plant_id %in% topN$plant_id)
        
          ggplot() +
            geom_sf(data=world, fill="grey97", color="grey85", size=0.2) +
            geom_sf(data=data_sub, aes(fill=centrality_score),
                    shape=16, size=2, stroke=0, alpha=0.5) +
            geom_sf(data=topN, aes(fill=centrality_score),
                    shape=21, size=7, stroke=0, alpha=0.3) +
            geom_sf(data=topN, aes(color=sector),
                    shape=21, size=7, stroke=2, fill=NA) +
        
            # unified legend
            geom_point(data=legend_df,
                       aes(x=x, y=y, color=sector),
                       shape=16, size=3, inherit.aes=FALSE) +
            geom_point(data=legend_df %>% filter(is_ring),
                       aes(x=x, y=y, color=sector),
                       shape=21, fill=NA, size=7, stroke=2,
                       inherit.aes=FALSE) +
        
            {if(zoom) geom_sf(data=cities_zoom, color="black", size=1.2)} +
            {if(zoom) geom_label_repel(data=cities_zoom,
                                       aes(label=NAME, geometry=geometry),
                                       stat="sf_coordinates",
                                       size=4, fill="white",
                                       box.padding=0.4, label.size=0.2)} +
        
            scale_fill_viridis_c(option="mako", direction=-1, limits=c(0,1),
                                 guide="none") +
            scale_color_manual(name="Top 50 offtakers / Sector",
                               values=sector_palette,
                               breaks=sector_levels,
                               drop=FALSE,
                               guide=if(show_legend) "legend" else "none") +
        
            coord_sf(xlim=zoom_xlim, ylim=zoom_ylim, expand=FALSE) +
            labs(title=title) +
            theme_classic(base_size=16) +
            theme(
              plot.title = element_text(hjust=0),
              legend.position = if(show_legend) "right" else "none",
              panel.border = element_rect(color="black", fill=NA, linewidth=0.6)
            )
        }

# Plots
        p1 <- plot_centrality_simple(offtakers_long, "degree",
                                     "Local Spillover Potential\n(Degree Centrality)",
                                     valleys_sf, additional_valleys_sf,
                                     bbox=bbox_zoom1)
        
        p2 <- plot_centrality_simple(offtakers_long, "betweenness",
                                     "Spillover Channels\n(Betweenness Centrality)",
                                     empty_valleys, empty_additional,
                                     show_legend=TRUE, bbox=bbox_zoom2)
        
        p3 <- plot_centrality_sector(offtakers_long, "degree",
                                     "North Rhine-Westphalia",
                                     valleys_sf, additional_valleys_sf,
                                     zoom=TRUE, show_legend=FALSE,
                                     zoom_xlim=zoom1_xlim, zoom_ylim=zoom1_ylim,
                                     cities_zoom=cities_zoom1)
        
        p4 <- plot_centrality_sector(offtakers_long, "betweenness",
                                     "Eastern Austria / Moravia",
                                     valleys_sf, additional_valleys_sf,
                                     zoom=TRUE, show_legend=TRUE,
                                     zoom_xlim=zoom2_xlim, zoom_ylim=zoom2_ylim,
                                     cities_zoom=cities_zoom2)
        
        edf3 <- (p1 + p2) / (p3 + p4) +
          plot_annotation(tag_levels="a",
                          theme=theme(plot.tag=element_text(face="bold", size=14))) &
          theme(
            legend.position="right",
            legend.box.spacing = unit(-25,"cm"),
            panel.spacing = unit(0,"cm"),
            axis.text = element_blank(),
            axis.title = element_blank(),
            axis.ticks = element_blank(),
            plot.margin = margin(0,0,0,0)
          )
        
        edf3 <- edf3 + plot_layout(
          guides="collect",
          widths=c(1,0.05)
        )
        
        options(repr.plot.width=24, repr.plot.height=12, repr.plot.res=600)
        print(edf3)
                               
        ggsave("extended-data-figure-3", edf3,
               device="png", width=24, height=12,
               units="in", dpi=600, limitsize=FALSE)